In [1]:
import tensorflow as tf
import numpy as np
import xlrd

model_inputs = np.array([])
model_outputs = np.array([])
test_inputs = np.array([])
test_outputs = np.array([])
test_score = np.array([])
model = tf.keras.models.Sequential()


In [2]:
def loadData(model_first_row, model_final_row, test_first_row, test_final_row):

    global model_inputs, model_outputs, test_inputs, test_outputs, test_score
    # Load Full dataset (masterData)
    ExcelFileName= 'Merval.xls'
    workbook = xlrd.open_workbook(ExcelFileName)
    worksheet = workbook.sheet_by_name("Data") 

    # model_first_row = 2500
    # model_final_row = 3500
    # test_first_row = 3501 
    # test_final_row = 3505

    score_col = 25
    first_col = 26
    final_col = 35

    model_inputs = np.zeros([model_final_row - model_first_row, final_col - first_col])
    model_outputs = np.zeros([model_final_row - model_first_row])

    for row in range(model_first_row - 1, model_final_row - 1):
        cell_value = worksheet.cell_value(row, first_col)
        model_outputs[row - model_first_row + 1] = cell_value

        for col in range(first_col, final_col):
            cell_value = worksheet.cell_value(row, col + 1)
            model_inputs[row - model_first_row + 1, col - first_col] = cell_value


    test_inputs = np.zeros([test_final_row - test_first_row, final_col - first_col])
    test_outputs = np.zeros([test_final_row - test_first_row])
    test_score = np.zeros([test_final_row - test_first_row])

    for row in range(test_first_row - 1, test_final_row - 1):
        cell_value = worksheet.cell_value(row, first_col)
        test_outputs[row - test_first_row + 1] = cell_value
        cell_value = worksheet.cell_value(row, score_col)
        test_score[row - test_first_row + 1] = cell_value

        for col in range(first_col, final_col):
            cell_value = worksheet.cell_value(row, col + 1)
            test_inputs[row - test_first_row + 1, col - first_col] = cell_value

            
def evaluate():
    
    global test_inputs, test_score, model
    wrong = 0.0
    right = 0.0
    none = 0.0
    results = []
    for r in range(len(test_score) ):
        predict = model.predict_classes(test_inputs[r:r+1])[0]
        results.append(predict)
        if ((predict==2 and test_score[r] > 0) or (predict==0 and test_score[r] < 0)):
            right += abs(test_score[r])
        if ((predict==2 and test_score[r] < 0) or (predict==0 and test_score[r] > 0)):
            wrong += abs(test_score[r])
        if (predict==1):
            none += abs(test_score[r])
    return wrong, right, none, results            


def optimize(tries):

    global model_inputs, model_outputs, model
    wrong = right = none = 0.0
    total_wrong = total_right = total_none = 0.0
    res = list()

    for t in range(tries):

        model = tf.keras.models.Sequential()  # a basic feed-forward model
        model.add(tf.keras.layers.Flatten())  # takes our 28x28 and makes it 1x784
        model.add(tf.keras.layers.Dense(3, activation=tf.nn.relu))  # a simple fully-connected layer, 128 units, relu activation
        model.add(tf.keras.layers.Dense(3, activation=tf.nn.relu))  # a simple fully-connected layer, 128 units, relu activation
        model.add(tf.keras.layers.Dense(3, activation=tf.nn.softmax))  # our output layer. 10 units for 10 classes. Softmax for probability distribution

        model.compile(optimizer='adam',  # Good default optimizer to start with
                      loss='sparse_categorical_crossentropy',  # how will we calculate our "error." Neural network aims to minimize loss.
                      metrics=['accuracy'])  # what to track

        model.fit(model_inputs, model_outputs, epochs=100, verbose=0)  # train the model

        results = []
        wrong, right, none, results = evaluate()
        total_wrong += wrong
        total_right += right
        total_none += none
        if (wrong + right == 0):
            result = 0
        else:
            result = 100*(right/(right + wrong))
        res.append(result)
        for t in range(len(results)):
            print("Projected Row:", end = '')
            print(projectedRow, end = ' ')
            print("Result:", end = '')
            print(projectedRow + t, end = ' ')
            print(results[t])

    print("{p:6.2f}".format(p=total_right), end = ' ')
    print("{p:6.2f}".format(p=total_wrong), end = ' ')
    print("{p:6.2f}".format(p=total_none), end = ' ')
    return total_right, total_wrong


In [ ]:
tot_r = tot_w = 0.0
opt_r = opt_w = 0.0
for projectedRow in range(1637, 4700, 1):
    loadData(projectedRow-250, projectedRow-1, projectedRow, projectedRow+10)
    opt_r, opt_w = optimize(1)
    tot_r += opt_r
    tot_w += opt_w
    print( "{p:6.2f}%".format(p=100*(tot_r/(tot_r+tot_w))))


Instructions for updating:
Colocations handled automatically by placer.
Projected Row:1637 Result:1637 2
Projected Row:1637 Result:1638 0
Projected Row:1637 Result:1639 0
Projected Row:1637 Result:1640 2
Projected Row:1637 Result:1641 2
Projected Row:1637 Result:1642 2
Projected Row:1637 Result:1643 2
Projected Row:1637 Result:1644 2
Projected Row:1637 Result:1645 2
Projected Row:1637 Result:1646 2
  4.83   5.50   0.00  46.75%
Projected Row:1638 Result:1638 2
Projected Row:1638 Result:1639 2
Projected Row:1638 Result:1640 2
Projected Row:1638 Result:1641 2
Projected Row:1638 Result:1642 2
Projected Row:1638 Result:1643 2
Projected Row:1638 Result:1644 2
Projected Row:1638 Result:1645 2
Projected Row:1638 Result:1646 2
Projected Row:1638 Result:1647 2
  4.23   5.84   0.00  44.41%
Projected Row:1639 Result:1639 2
Projected Row:1639 Result:1640 2
Projected Row:1639 Result:1641 2
Projected Row:1639 Result:1642 2
Projected Row:1639 Result:1643 2
Projected Row:1639 Result:1644 2
Projected Ro